import library

In [327]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

membuat dataframe dari datatrain format csv

In [328]:
datatrain = pd.read_csv('datatrain.csv',sep=';',usecols = ['11','258','260','264','294'])

In [329]:
datatrain

,11,258,260,264,294
0,2,1,0,1,2
1,2,0,3,1,0
2,3,0,1,0,1
3,0,5,3,3,1
4,5,2,0,3,1
5,0,0,5,3,3
6,5,4,4,0,2
7,4,0,4,0,2


dari dataframe diubah bentuk menjadi array

In [330]:
dataarray = datatrain.values.tolist()
print(dataarray)

[[2, 1, 0, 1, 2], [2, 0, 3, 1, 0], [3, 0, 1, 0, 1], [0, 5, 3, 3, 1], [5, 2, 0, 3, 1], [0, 0, 5, 3, 3], [5, 4, 4, 0, 2], [4, 0, 4, 0, 2]]


fungsi kmeans

In [331]:
def main(data,k): #fungsi inti kmeans
    centroids = (dataarray[0],dataarray[1]) #memanggil centroid dari variabel dataarray 

    while True:
        old_centroids = centroids
        label = get_labels(data,centroids) #memanggil fungsi getlabels
        centroids = update_centroids(data,label,k) #memanggil fungsi updatecentroid
        # print([old_centroids,label,centroids])
        if should_stop(old_centroids,centroids): #syarat iterasi berhenti
            break
    return label #hasil berupa label cluster 0 dan 1

def get_labels(data,centroids): #untuk menandai cluster terdekat
    labels = []
    for point in data:
        min_dist = float('inf')
        label = None
        for i, centroid in enumerate(centroids):
            new_dist = get_distance(point,centroid) #memanggil fungsi getdist untuk melihat nilai eucdist
            if min_dist > new_dist: #jika dist kecil maka dipilih
                min_dist = new_dist
                label = i
        labels.append(label)
    return labels

def get_distance(point_1,point_2): #menghitung eucdist 
    distance = (((point_1[0] - point_2[0]) ** 2) + ((point_1[1] - point_2[1]) ** 2) + ((point_1[2] - point_2[2]) ** 2) + ((point_1[3] - point_2[3]) ** 2) + ((point_1[4] - point_2[4]) ** 2)) ** 0.5 
    return distance   

def update_centroids(points,labels,k): #fungsi untuk update nilai centroid
    new_centroids = [[0,0,0,0,0] for i in range(k)] #inisialisasi new centroid = 0
    counts = [0]*k 
    for point,label in zip(points,labels):
        new_centroids[label][0] += point[0]
        new_centroids[label][1] += point[1]
        new_centroids[label][2] += point[2]
        new_centroids[label][3] += point[3]
        new_centroids[label][4] += point[4]
        counts[label] += 1

    for i, (a,b,c,d,e) in enumerate(new_centroids):
        new_centroids[i] = (a/counts[i],b/counts[i],c/counts[i],d/counts[i],e/counts[i])
    return new_centroids

def should_stop(old_centroids,new_centroids): #berhenti jika nilai centroid tidak berubah
    return old_centroids == new_centroids

data = dataarray
k = 2
main(data,k)

[0, 1, 0, 0, 0, 1, 1, 1]

Data label dari kmeans dimasukkan ke variabel tiap cluster

In [332]:
hasilkmeans = main(data,k) #label dari fungsi kmeans ditampung pada variabel
cluster1 = []
cluster2 = []
for i, j in enumerate(hasilkmeans):
    if j == 1:
        cluster2.append(dataarray[i])
    else:
        cluster1.append(dataarray[i])
print('cluster 1 :',cluster1)
print('cluster 2 :',cluster2)

cluster 1 : [[2, 1, 0, 1, 2], [3, 0, 1, 0, 1], [0, 5, 3, 3, 1], [5, 2, 0, 3, 1]]
cluster 2 : [[2, 0, 3, 1, 0], [0, 0, 5, 3, 3], [5, 4, 4, 0, 2], [4, 0, 4, 0, 2]]


In [333]:
def avg(cluster): #fungsi u/ menghitung average
    averageclust = []
    for i in cluster: 
        count = 0
        avge = 0
        for j in i:
            if j != 0:
                avge += j #jumlah nilai pada baris cluster
                count += 1 #len baris cluster
        averageclust.append(avge/count)
    return averageclust
averageclust1 = avg(cluster1)
averageclust2 = avg(cluster2)
print('average cluster 1 :',averageclust1)
print('average cluster 2 :',averageclust2)

average cluster 1 : [1.5, 1.6666666666666667, 3.0, 2.75]
average cluster 2 : [2.0, 3.6666666666666665, 3.75, 3.3333333333333335]


In [334]:
def mcrcluster(cluster,average): #fungsi u/ menghitung mcr
    mcrclust = []
    for i in range(len(cluster)):
        data = []
        for j in range(len(cluster[i])):
            if cluster[i][j] != 0:
                data.append(cluster[i][j]-average[i])
            else:
                data.append(cluster[i][j]) #jika nilai cluster 0 maka dibiarkan/tidak dihitung
        mcrclust.append(data)
    return mcrclust
mcrclust1 = mcrcluster(cluster1,averageclust1)
mcrclust2 = mcrcluster(cluster2,averageclust2)
print('mcr cluster 1 :',mcrclust1)
print('mcr cluster 2 :',mcrclust2)

mcr cluster 1 : [[0.5, -0.5, 0, -0.5, 0.5], [1.3333333333333333, 0, -0.6666666666666667, 0, -0.6666666666666667], [0, 2.0, 0.0, 0.0, -2.0], [2.25, -0.75, 0, 0.25, -1.75]]
mcr cluster 2 : [[0.0, 0, 1.0, -1.0, 0], [0, 0, 1.3333333333333335, -0.6666666666666665, -0.6666666666666665], [1.25, 0.25, 0.25, 0, -1.75], [0.6666666666666665, 0, 0.6666666666666665, 0, -1.3333333333333335]]


Pearson Similarity

In [335]:
def funKiri(datakiri):
    hasil = 0
    for i in datakiri:
        hasil += i
    return hasil #sum data kiri

def funKanan(datakanan):
    hasil = 0
    baris1 = []
    baris2 = []
    # print(datakanan[0] , datakanan[2], (datakanan[0] + datakanan[2])**0.5)
    for i, data in enumerate(datakanan):
        if i % 2 == 0:
            baris1.append(data)
        else:
            baris2.append(data)
    hasil = (sum(baris1)**0.5) * (sum(baris2)**0.5) #sum data kanan diakar lalu dijumlah
    return hasil
            
            
def similariti(mcr):
    sim = []
    for i,arr in enumerate(mcr): #ambil data user dari mcr
        case = []
        # print(i, arr)
        for j in range(i+1,len(mcr)): #loop perbandingan antar user
            datakiri = []
            datakanan = []
            for x in range(len(arr)): #loop hitung data similatiras
                if isinstance(arr[x], float) and isinstance(mcr[j][x], float) == True: #nilai 0 integer tidak dihitung
                    datakiri.append(arr[x] * mcr[j][x]) #data sebelah kiri untuk dihitung pada funkiri
                    datakanan.append(mcr[j][x] ** 2) #data sebelah kanan untuk dihitung pada funkanan
                    datakanan.append(arr[x] ** 2)
                
            hasilkiri = funKiri(datakiri) #data dibagi ke sisi kanan dan kiri untuk mempermudah alur
            hasilkanan = funKanan(datakanan)
            # print(hasilkiri/hasilkanan)
            case.append([j,hasilkiri/hasilkanan]) #data kiri dan kanan dibagi lalu diappend ke var sim
        sim.append([i,case])
    return sim
simclust1 = similariti(mcrclust1)
simclust2 = similariti(mcrclust2)
print(simclust1)
print(simclust2)

[[0, [[1, 0.31622776601683783], [2, -0.8164965809277259], [3, 0.1690308509457033]]], [1, [[2, 0.7071067811865476], [3, 0.9805806756909203]]], [2, [[3, 0.3682298471593294]]], [3, []]]
[[0, [[1, 0.9486832980505138], [2, 0.19611613513818404], [3, 0.7071067811865476]]], [1, [[2, 0.5692099788303083], [3, 0.7999999999999999]]], [2, [[3, 0.9428090415820632]]], [3, []]]


Perhitungan prediksi rating dengan user-based

In [336]:
datatest = pd.read_csv('datatest.csv',sep=';',usecols = ['userid','11','258','260','264','294'])
datatest

,userid,11,258,260,264,294
0,1,0,0,1,0,0
1,2,0,0,0,0,1
2,3,0,0,0,1,0
3,4,5,0,0,0,0
4,6,0,0,2,0,0
5,7,2,0,0,0,0
6,8,0,0,0,1,0
7,10,0,0,0,0,0


In [357]:
#index item yg diprediksi sama dengan index datatest
# ambil user yang merating
test = datatest.values.tolist()
def maxSim(data): #mencari 2 nilai tertinggi dari sim
    test = []
    for i in range(len(data)):
        test.append([data[i][0],abs(data[i][1])])
    test.remove(min(test))
    print(test)

hasilTest = []
userC1 = 0 #untuk menandai user pada cluster 1
userC2 = 0 #untuk menandai user pada cluster 2 karena tidak ada indexnya
for i,data in enumerate(test): #loop semua user
    for j in range(1,len(data)): #lopp data pada user selain id user
        if hasilkmeans[i] == 0: #Menentukan Usercluster 1 / 2
            avg = averageclust1
            sim = simclust1
            mcr = mcrclust1
            if data[j] != 0: #menentukan data yang dirating
                dataClus = [x for x in range(len(cluster1)) if cluster1[x][j-1] != 0] #mencari data cluster yang tidak 0
                if len(dataClus) >= 2:
                    print(dataClus)
                    maxSim(sim[userC1][1])
                # ngitung = avg[userC1]*(((sim[userC1]*mcr[1][2])+(sim[userC2]*mcr[2][2]))/(abs(sim[userC1])+abs(sim[userC2])))
                userC1 += 1
        else:
            avg = averageclust1
            sim = simclust1
            mcr = mcrclust1
            if data[j] != 0: #menentukan data yang dirating
                dataClus = [x for x in range(len(cluster1)) if cluster1[x][j-1] != 0]
                # if len(dataClus) >= 2:
                    # print(dataClus)
                    # maxSim(sim[userC1][1])
                # ngitung = avg[userC1]*(((sim[userC1]*mcr[1][2])+(sim[userC2]*mcr[2][2]))/(abs(sim[userC1])+abs(sim[userC2])))




[1, 2]
[[2, 0.8164965809277259], [3, 0.1690308509457033]]
[0, 2, 3]
[[3, 0.9805806756909203]]
[0, 1, 3]
[]
[1, 2]


ValueError: min() arg is an empty sequence

In [338]:
r1260 = averageclust1[0]*(((simclust1[0]*mcrclust1[1][2])+(simclust1[1]*mcrclust1[2][2]))/(abs(simclust1[0])+abs(simclust1[1]))) 
r2294 = averageclust2[0]*(((simclust2[0]*mcrclust2[1][4])+(simclust2[1]*mcrclust2[2][4]))/(abs(simclust2[0])+abs(simclust2[1]))) 
r3264 = averageclust1[1]*(((simclust1[4]*mcrclust1[3][3])+(simclust1[3]*mcrclust1[3][2]))/(abs(simclust1[4])+abs(simclust1[3]))) 
r411 = averageclust1[2]*(((simclust1[3]*mcrclust1[1][0])+(simclust1[5]*mcrclust1[3][0]))/(abs(simclust1[3])+abs(simclust1[5]))) 
r6260 = averageclust1[3]*(((simclust1[4]*mcrclust1[1][2])+(simclust1[5]*mcrclust1[2][2]))/(abs(simclust1[4])+abs(simclust1[5]))) 
r711 = averageclust2[1]*(((simclust2[0]*mcrclust2[0][0])+(simclust2[4]*mcrclust2[3][0]))/(abs(simclust2[0])+abs(simclust2[4]))) 
r8264 = averageclust2[2]*(((simclust2[3]*mcrclust2[1][3])+(simclust2[1]*mcrclust2[0][3]))/(abs(simclust2[3])+abs(simclust2[1]))) 
print(r1260,r3264,r411,r6260)
print(r2294,r711,r8264)

TypeError: can't multiply sequence by non-int of type 'float'

Evaluasi dengan RMSE fold 1

In [ ]:
rmse = ((((1-r1260)**2)+((2-r2294)**2)+((1-r3264)**2)+((5-r411)**2)+((2-r6260)**2)+((2-r711)**2)+((1-r8264)**2))**0.5)/7
print(rmse)

0.9304092792665741
